In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import itertools as it
import matplotlib.pyplot as pl
from scipy import stats, linalg

import multiprocessing as mp
import time

from thresholding import *
import kneeliverse.rdp as rdp
import kneeliverse.kneedle as kneedle

#from castle.common import independence_tests

In [2]:
def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p))
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
        
    return P_corr

In [4]:
#ETL
df = pd.read_csv("notas-alunos-2012-2022-anonimo.csv")
total_len=len(df)

df.drop(df[df.in_mobility==1].index,axis=0,inplace=True)
df.drop(["in_mobility","mobility_start","mobility_duration"],axis=1,inplace=True)
df.drop(df[(df.access_type == "Other_8")].index,axis=0,inplace=True)

df = df[df.regime_type == "Normal"] #84%
df.drop("regime_type",axis=1,inplace=True)
print("Total kept:", np.round(len(df)/total_len,2) )

df.drop(df[df.exam_type=="December"].index,axis=0,inplace=True)
df.drop(df[df.exam_type=="Semestral"].index,axis=0,inplace=True)
df.drop(df[df.exam_type=="Special"].index,axis=0,inplace=True)
df.drop(df[df.exam_season=="Special"].index,axis=0,inplace=True)

print("Total kept:", np.round(len(df)/total_len,2) )

df['istudent'] = df['istudent'].astype(str)

df.drop(df[df.was_evaluated==0].index,axis=0,inplace=True) #non-evaluated
df.drop(df[df.grade==0].index,axis=0,inplace=True) #removing zeros

print("Total kept:", np.round(len(df)/total_len,2) )

df.drop(df[df.exam_season == "Retake"].index,axis=0,inplace=True) #recursos

print("Total kept:", np.round(len(df)/total_len,2) )

Total kept: 0.83
Total kept: 0.82
Total kept: 0.61
Total kept: 0.5


In [8]:
#Histogram
def histogramer(curso,col1,col2,i_,save):

    x,y = valid_vals(X, col1, col2)[0].T
    pl.plot(x,y,'.',alpha=0.4,label="Points: %i"%len(x))
    m,d,r2=stats.linregress(x,y)[:3]
    pl.plot(np.array([0,20]),np.array([0,20])*m+d,'--',label="Fit: %.2f"%r2)
    
    pl.title("Degree "+str(curso))
    pl.xlabel("Curricular Unit "+str(col1))
    pl.ylabel("Curricular Unit "+str(col2))
    pl.grid()
    pl.ylim(0-0.5,20+0.5)
    pl.xlim(0-0.5,20+0.5)
    pl.legend()

    if save:
        pl.savefig("figures/neg_"+str(i_))
    pl.show()

def valid_vals(X, col1, col2):
    vals = []
    studs= []
    Xpair = (lambda dd: dd[dd.notna().all(axis=1)] )(X[[col1,col2]])
    for p in Xpair.index:
        d1,d2=Xpair.loc[p]
                    
        # Use efficient search for minimum positive difference
        diffs = np.array(list(it.product(d1.keys(), d2.keys())))
        pos_diffs = diffs[:, 1] - diffs[:, 0]
        mask = pos_diffs > 0

        if not np.any(mask):  # Skip if no positive differences exist
            continue
        
        best_idx = np.argmin(pos_diffs[mask])
        best_a, best_b = diffs[mask][best_idx]
        # Append the corresponding values to vals
        vals.append([d1[best_a], d2[best_b]])
        studs.append(p)

    return np.array(vals),np.array(studs)

In [10]:
save = False

anticorr_total = []
anticorr_signi = []

i_ = 0
for curso in np.unique(df.idegree):
    dg = df[df.idegree==curso].copy()

    B = nx.Graph()
    B.add_nodes_from(dg['icourse'].unique(),bipartite=0)
    B.add_nodes_from(dg['istudent'].unique(), bipartite=1)
    B.add_edges_from(dg[['istudent','icourse']].values.tolist())
    
    G = nx.bipartite.weighted_projected_graph(B,dg['icourse'].unique())

    n_weights = [ (dg.icourse == disc).sum()*0.7 for disc in G.nodes]

    edges, weights = zip(*nx.get_edge_attributes(G, 'weight').items())
    e_weights = [w * 0.1 for w in weights]

    if save:
        dg.nota.hist(bins=np.arange(21),color='skyblue',edgecolor='black')
        pl.xticks(np.arange(21))
        pl.xlim(0,20)
        #pl.ylim(0,100000)
        pl.grid(axis="x")
        pl.title("Distribution of scores in evaluated subjects")
        pl.xlabel("Scores")
        pl.savefig("figures/"+str(curso)+"_hist_scores")
        pl.show()
    

    cut_stud = 20
    H = G.edge_subgraph([(a,b) for a, b, attrs in G.edges(data=True) if attrs["weight"] >= cut_stud]).copy()
    
    comp_list = sorted(nx.connected_components(H), key=len, reverse=True)
    
    #LCC
    if len(H.nodes()) == 0:
        continue
    H = H.subgraph(list(nx.connected_components(H))[0])
    
    n_weights = [ (dg.icourse == disc).sum()*0.5 for disc in H.nodes]
    
    edges, weights = zip(*nx.get_edge_attributes(H, 'weight').items())
    e_weights = [w * 0.005 for w in weights]
    
    if save:
        pl.figure(figsize=(16,16))
        nx.draw(H, node_size=n_weights, width=e_weights, edgecolors='black')
        pl.savefig("figures/"+str(curso)+"_network_cutoff20_LCC")
        pl.show()

    # Initialize shared memory for the results
    course_semester = mp.Manager().dict()
    
    # Define function to process each discipline
    def process_disc(disc):
        temp_ = np.sort(dg[dg.icourse == disc].exam_season.unique())
        
        # Define rules and add results to the dictionary
        if temp_[0] == "Annual":
            return disc, 2, None  # Second semester for annual course
       # elif temp_[0] == "Retake":
       #     return disc, None, None  # Ignore 'Retake' courses
        elif "1st Semester" in temp_ and "2nd Semester" in temp_:
            dg.loc[(dg.icourse == disc) & (dg.exam_season == "2nd Semester"), "icourse"] = -disc
            return -disc, 2, disc  # New discipline with negative numbering
        
        return disc, int(temp_[0][0]), None  # Set semester based on first character of temp_
    
    # Parallel execution
    with mp.Pool() as pool:
        results = pool.map(process_disc, dg.icourse.unique())
    
    # Update course_semester and dg with the results
    for disc, semester, updated_disc in results:
        if semester is not None:
            course_semester[disc] = semester
        if updated_disc is not None:
            course_semester[updated_disc] = 2  # Mark as second semester
    
    course_semester = dict(course_semester)

    entry_year = dg.groupby('istudent')['year'].min().to_dict()


    # Function to process each student's records
    def process_student(student_group):
        student, group = student_group
        entry_year_student = entry_year[student]
        
        # Initialize courses dictionary for this student
        courses = {col: None for col in course_semester.keys()}
    
        # Iterate over records for this student
        for _, row in group.iterrows():
            disc = row['icourse']
            
            if disc not in courses:
                continue
            
            # Calculate semester
            semt = (row['year'] - entry_year_student) * 2 + (course_semester[disc] - 1)
            
            current_grade = row['grade']
            
            # Update grades only if valid and lower than the stored grade
            if courses[disc] is not None:
                if semt in courses[disc]:
                    if courses[disc][semt] > current_grade > 0:
                        courses[disc][semt] = current_grade
                elif current_grade > 0:
                    courses[disc][semt] = current_grade
            elif current_grade > 0:
                courses[disc] = {semt: current_grade}
                
        return student, courses
    
    # Creating the DataFrame `X` in parallel
    def create_X_dataframe(df):
        # Initialize an empty DataFrame with the required shape and columns
        X = pd.DataFrame(index=df['istudent'].unique(), columns=course_semester.keys())
        
        # Split data by 'istudent'
        student_groups = list(df.groupby('istudent'))
        
        # Use multiprocessing Pool to process each student in parallel
        with mp.Pool(mp.cpu_count()) as pool:
            results = pool.map(process_student, student_groups)
        
        # Populate the DataFrame `X` with results from the parallel processing
        for student, courses in results:
            X.loc[student] = courses
        
        return X
    
    # Call function to create the DataFrame
    X = create_X_dataframe(dg)
    X = X.dropna(axis="columns",how="all")
    X = X.dropna(axis="index",how="all")


    
    ti = time.time()
    
    corr = {}
    n_stud = {}
    
    valid_pairs = set(H.edges)
    for col1,col2 in it.permutations(X.columns,2):
    
        #Skip if edge is not in H
        if (abs(col1), abs(col2)) not in valid_pairs and (abs(col2), abs(col1)) not in valid_pairs:
            continue
    
        vals=valid_vals(X, col1, col2)[0]
            
        if len(vals)<max(2,cut_stud): #cut-off min alunos #Note that it might leave the graph disconnected
            continue
    
        if np.any(np.var(vals,axis=0)==0):
            continue
        
        corr[(col1, col2)] = np.corrcoef(vals[:, 0], vals[:, 1])[0, 1]
        #corr[(col1,col2)] = independence_tests.CITest.fisherz_test(vals,0,1,[])[2]
        n_stud[(col1, col2)] = len(vals)

    corr_vals = np.abs(list(corr.values()))
    corr_vars = np.array(list(corr))
    
    corr_vals,corr_vars = np.sort(corr_vals), corr_vars[np.argsort(corr_vals)]
    #corr_vars, corr_vals = np.flip(corr_vars,axis=0), np.flip(corr_vals,axis=0)
    
    # Initialize the graph with all edges
    K_ = nx.Graph()
    K_.add_edges_from(corr_vars)
    # Create a subgraph of the largest connected component
    if len(K_.nodes())==0:
        continue
    K = K_.subgraph(sorted(nx.connected_components(K_), key=len, reverse=True)[0]).copy()
    corr_vars_ = corr_vars[np.array([i for i in range(len(corr_vars)) if corr_vars[i] in K.edges])]
    corr_vals_ = corr_vals[np.array([i for i in range(len(corr_vars)) if corr_vars[i] in K.edges])]
    
    #Connected Threshold
    m = binary_search(np.unique(corr_vars_),corr_vars_)
    m1=m
    
    #'''
    #Knee Threshold
    gcc_nodes=np.zeros(len(corr_vars_))
    for j in range(len(corr_vars_)-1):
        # Get the size of the largest connected component
        gcc_nodes[j] = len(max(nx.connected_components(K),key=len))
    
        jT = np.where(np.all(np.flip(corr_vars_[j]) == corr_vars_,axis=1))[0]
        if len(jT)>0 and corr_vals_[j]>corr_vals_[jT]:
            continue
        
        # Remove the edge corresponding to the current step
        K.remove_edge(*corr_vars_[j])
        
    m = kneedle.knee(np.column_stack((np.arange(len(gcc_nodes)),gcc_nodes)))
    thres = corr_vals[m]
    
    if not m:
        continue
    #'''
    
    
    if save:
        pl.plot(gcc_nodes,'.')
        pl.plot(m1,gcc_nodes[m1],'*')
        pl.plot(m,gcc_nodes[m],'r*')
        pl.grid()
        pl.savefig("figures/"+str(curso)+"_kneegraph")
        pl.show()

    if save:
        pl.hist(corr.values(),bins=np.linspace(-1,1,17),color='skyblue',edgecolor='black')
        pl.title("Distribution of correlation coeficients")
        pl.xlim(-1,1)
        ylims = pl.gca().get_ylim()
        pl.plot(np.array([-thres,-thres]),ylims,'r--',label="Knee")
        pl.plot(np.array([thres,thres]),ylims,'r--')
        pl.plot(np.array([-corr_vals[m1],-corr_vals[m1]]),ylims,'g--',label="Connected")
        pl.plot(np.array([corr_vals[m1],corr_vals[m1]]),ylims,'g--')
        pl.grid(axis="y")
        pl.legend(loc=2)
        pl.savefig("figures/"+str(curso)+"_corrhist")
        pl.show()

    ti = time.time()
    
    G1 = nx.DiGraph()
    G1.add_edges_from(corr_vars[m:])

    G0 = nx.DiGraph()
    G0.add_edges_from(corr_vars)

    #Triangulation
    '''
    removed=[]
    
    for col1 in np.array(G1.nodes)[np.array(G1.in_degree)[:,1].astype("int")>1]: #anchoer nodes with in-degree greater than 1
        
        parents = np.array(list(G1.in_edges(col1)))[:,0]
    
        for col2,col3 in it.permutations(parents,2):
            
            if (col2,col3) not in G.edges or (col3,col2) not in G.edges:
                continue
            
            vals = []
            for d1,d2,d3 in (lambda dd: dd[dd.notna().all(axis=1)] )(X[[col1,col2,col3]]).values:
    
                # Use efficient search for minimum positive difference
                diffs12 = np.array(list(it.product(d1.keys(), d2.keys())))
                pos_diffs12 = diffs12[:, 1] - diffs12[:, 0]
                mask12 = pos_diffs12 > 0
    
                diffs13 = np.array(list(it.product(d1.keys(), d3.keys())))
                pos_diffs13 = diffs13[:, 1] - diffs13[:, 0]
                mask13 = pos_diffs13 > 0
    
                if not np.any(mask12) or not np.any(mask13):  # Skip if no positive differences exist
                    continue
    
                best_a, best_b = diffs12[mask12][np.argmin(pos_diffs12[mask12])]
                best_d, best_c = diffs13[mask13][np.argmin(pos_diffs13[mask13])]
    
                if best_a != best_d: #noncoincident semesters: no common cause found
                    continue
    
                # Append the corresponding values to vals
                vals.append([d1[best_a], d2[best_b], d3[best_c]])
            
            if len(vals)<=cut_stud: #cut-off min alunos
                continue
                
            vals = np.array(vals)
            
            if np.any(np.var(vals,axis=0)==0):
                continue
    
            partial_corrs = partial_corr (vals)
            if (col2,col1) in G1.edges and np.abs(partial_corrs[0,1]) < thres:
                G1.remove_edge(col2,col1)
                removed.append((col2,col1,col3))
                
            if (col3,col1) in G1.edges and np.abs(partial_corrs[0,2]) < thres:
                G1.remove_edge(col3,col1)
                removed.append((col3,col1,col2))
    #'''
        
    dres = pd.DataFrame(corr_vars[:],columns=["Var1","Var2"])
    dres["corr_val"] = corr_vals[:]
    dres["StudentBoth"] = [n_stud[(dres.loc[i]["Var1"],dres.loc[i]["Var2"])] for i in dres.index]
    
    dict_semmode = {colx: stats.mode(sum([list(ii.keys())
                                          for ii in X[colx][X[colx].notna()] ],[]))[0]
                    for colx in X.columns}
    dres["SemesterMode1"] = [dict_semmode[dres["Var1"].loc[i]] for i in dres.index]
    dres["SemesterMode2"] = [dict_semmode[dres["Var2"].loc[i]] for i in dres.index]
    
    dict_semfirsttry = {colx: stats.mode([list(ii.keys())[0]
                                          for ii in X[colx][X[colx].notna()] ])[0]
                    for colx in X.columns}
    dres["SemesterFirstTry1"] = [dict_semfirsttry[dres["Var1"].loc[i]] for i in dres.index]
    dres["SemesterFirstTry2"] = [dict_semfirsttry[dres["Var2"].loc[i]] for i in dres.index]
    
    dict_semfirsttry_ave = {colx: np.mean([list(ii.keys())[0]
                                          for ii in X[colx][X[colx].notna()] ])
                    for colx in X.columns}
    dres["SemesterFirstTry1_Mean"] = [dict_semfirsttry_ave[dres["Var1"].loc[i]] for i in dres.index]
    dres["SemesterFirstTry2_Mean"] = [dict_semfirsttry_ave[dres["Var2"].loc[i]] for i in dres.index]
    
    
    dict_semmean = {}
    for colx in X.columns:
        # Combine all keys from non-None dictionaries in the column
        keys = sum([list(ii.keys()) for ii in X[colx][X[colx].notna()]], [])
        
        # Compute the mean if keys exist; otherwise, assign np.nan
        if keys:
            dict_semmean[colx] = np.mean(keys)
        else:
            dict_semmean[colx] = np.nan
    dres["SemesterMean1"] = [dict_semmean[dres["Var1"].loc[i]] for i in dres.index]
    dres["SemesterMean2"] = [dict_semmean[dres["Var2"].loc[i]] for i in dres.index]
    
    dict_mean = {colx: np.round(np.mean(sum([list(ii.values())
                                             for ii in X[colx][X[colx].notna()] ],[])),2)
                 for colx in X.columns}
    dres["Mean1"] = [dict_mean[dres["Var1"].loc[i]] for i in dres.index]
    dres["Mean2"] = [dict_mean[dres["Var2"].loc[i]] for i in dres.index]
    
    dict_course = {colx:list(np.unique(dg.idegree[dg.icourse==abs(colx)])) for colx in X.columns}
    dres["Courses1"] = [dict_course[dres["Var1"].loc[i]] for i in dres.index]
    dres["Courses2"] = [dict_course[dres["Var2"].loc[i]] for i in dres.index]
    
    dict_years = {colx:list(np.unique(dg.year[dg.icourse==abs(colx)])) for colx in X.columns}
    dres["Years1"] = [dict_years[dres["Var1"].loc[i]] for i in dres.index]
    dres["Years2"] = [dict_years[dres["Var2"].loc[i]] for i in dres.index]

    temp1_,temp2_ = np.unique(dres["Var1"],return_index=True)
    order = {temp1_[i]:dres.loc[temp2_[i]].SemesterFirstTry1 for i in range(len(temp1_))}
    
    temp1_,temp2_ = np.unique(dres["Var2"],return_index=True)
    for i in range(len(temp1_)):
        order[temp1_[i]] = dres.loc[temp2_[i]].SemesterFirstTry2

    G_ = nx.DiGraph()
    G_.add_edges_from(corr_vars[m:])

    
    node_list = np.array(list(order))[np.argsort(list(order.values()))]
    
    heat = np.zeros([len(node_list),len(node_list)])
    for i in range(len(node_list)):
        for j in range(len(node_list)):
            if (node_list[i],node_list[j]) in corr:
                
                if np.abs(corr[(node_list[i],node_list[j])]) < thres: continue
                    
                #heat[i,j] = 1-corr[(node_list[i],node_list[j])]
                heat[i,j] = corr[(node_list[i],node_list[j])]
                #heat[j,i] = 1#heat[i,j]


    if save:
        fig, ax = pl.subplots(figsize=(16,20))
        pl.imshow(heat, cmap='RdGy', interpolation='nearest',vmin=-1, vmax=1)
        pl.colorbar(shrink=0.65)
        
        pl.plot(np.arange(len(node_list)+1)-0.5,
                np.arange(len(node_list)+1)-0.5,'k--')
        #pl.grid()
        
        ax.xaxis.tick_top()
        pl.xticks(range(len(node_list)),[a if a in cod2name else a for a in node_list],rotation=90,size=10)
        pl.yticks(range(len(node_list)),[a if a in cod2name else a for a in node_list],rotation=20,size=10)
        
        for i in np.cumsum(np.unique(list(order.values()),return_counts=True)[1])[:-1]:
            pl.plot(np.arange(len(node_list)+1)-0.5,
                    np.zeros(len(node_list)+1)+i-0.5,'k-')
            pl.plot(np.zeros(len(node_list)+1)+i-0.5,
                    np.arange(len(node_list)+1)-0.5,'k-')
        
        pl.savefig("figures/"+str(curso)+"_heatmap.png",dpi=300)
        pl.show()

    T = nx.DiGraph()
    for i in G0.nodes:
        T.add_node(i,layer=order[i])
    
    T.add_edges_from(G1.edges())
    

    if save:
        pl.figure(figsize=(16,16))
        pos = nx.multipartite_layout(T, subset_key="layer")
        for node in pos: #introduce a shift in the x axis
            pos[node] = pos[node] + np.array([np.random.random()*0.02-0.01,0])

        color_map = ["tab:red" if (node in G0.nodes - G1.nodes) else "tab:blue" for node in T.nodes]
        color_edg = ["tab:red" if corr[edge]<0 else "k" for edge in T.edges]
        width = [ {(dres["Var1"].loc[i],dres["Var2"].loc[i]):dres["StudentBoth"].loc[i]/300 for i in dres.index}[edge] for edge in G1.edges ]
        nx.draw_networkx_nodes(T,pos=pos, node_color=color_map)
        nx.draw_networkx_edges(T,pos=pos,alpha=np.abs(corr_vals[m:]),
                               edge_color=color_edg,width=width,arrowsize=20) #width scalled 1/300
        nx.draw_networkx_labels(T,pos=pos)
        pl.savefig("figures/"+str(curso)+"_graph_sem.png")
        pl.box(False)
        pl.show()

    #print("total edges:\t",len(T.edges))
    #print("doubles edges:\t",int(len([edge for edge in T.edges if np.flip(edge) in T.edges])/2))
    
    [edge for edge in T.edges if np.flip(edge) in T.edges]

    anticorr_total.append( np.sum(np.array(list(corr.values()))<0) )
    anticorr_signi.append( np.sum(heat<0) )

    
    if np.any(heat<0):
        for pair in node_list[np.array(np.where(heat<0)).T]:
            histogramer(curso,pair[0],pair[1],i_,save)
            i_+=1

anticorr_total = np.array(anticorr_total)
anticorr_signi = np.array(anticorr_signi)